In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model

In [ ]:
# Load the IMDB dateset word index
word_index=imdb.get_word_index()
reverse_word_index={value:key for key,value in  word_index.items()}

In [ ]:
# load the pre-trained model with RelU activation
model=load_model('simple_rnn_imdb.h5')

In [ ]:
model.summary()

In [ ]:
# Gives us all the weights
model.get_weights()

In [ ]:
# step2:Helper functions
# Function to decode reviews
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i-3,'?') for i in encoded_review])

# Function to preprocess user input
def preprocess_text(text):
    words=text.lower().split()
    encoded_review = []
    for word in words:
        index = word_index.get(word, 2) + 3  # +3 for IMDB offset
        if index >= 10000:  # outside vocabulary
            index = 2  # map to <UNK> token
        encoded_review.append(index)
    padded_review=sequence.pad_sequences([encoded_review],maxlen=500)
    return padded_review

In [ ]:
# Prediction function
def predict_sentiment(review):
    preprocessed_input=preprocess_text(review)
    prediction=model.predict(preprocessed_input)
    sentiment='Positive' if prediction[0][0] >0.5 else 'Negative'
    return sentiment,prediction[0][0]

In [ ]:
# Example Review for prediction
example_review="The movie was fantastic! The acting wa great and the plot was thrilling"
sentiment,score=predict_sentiment(example_review)
print(f"Review :{example_review}")
print(f"Sentiment :{sentiment}")
print(f"Prediction score: {score}")